In [3]:
import os 
import sys
import getopt
import pandas as pd
# import MySQLdb as mdb
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
import re
import shelve
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [4]:
resultsdir = './results/regional_income/'

manf = []
rettrd = []

for fn in os.listdir(resultsdir):
    # if os.path.isfile(fn):
    m_manf = re.search('results.*manf.*\.shelf',fn)
    if m_manf:
        
        shelf = shelve.open(resultsdir+fn, 'r')
        args_dict = shelf['args_dict']
        shelf.close()
        
        manf.append(args_dict)
    else:
        m_rettrd = re.search('results.*rettrd.*\.shelf',fn)
        if m_rettrd:

            shelf = shelve.open(resultsdir+fn, 'r')
            args_dict = shelf['args_dict']
            shelf.close()
            
            rettrd.append(args_dict)

In [38]:
def dict_to_dfs(args_dict):
    df_dict = {}
    fill_dict = {}
    fill_dict['attributes'] = {}
    fill_dict['scores'] = {}
    for key in args_dict:
        print key
        if type(args_dict[key]) == type([]) or type(args_dict[key]) == type(np.array([])):
            fill_dict['attributes']['n_%s'%key] = [len(args_dict[key])]
            
            fill_dict[key] = {}
            fill_dict[key]['geofips'] = [args_dict['geofips']]
            fill_dict[key]['industry'] = [args_dict['industry']]
            
            for name,value in zip(['val%d'%i for i in range(0,len(args_dict[key]))],args_dict[key]):
                fill_dict[key][name] = value
            
        # for dictionary entries (i.e. the fit scores)
        elif type(args_dict[key]) == type({}):
            if 'geo' not in fill_dict['scores']:
                fill_dict['scores']['geofips'] = []
            if 'industry' not in fill_dict['scores']:
                fill_dict['scores']['industry'] = []
            if 'case' not in fill_dict['scores']:
                fill_dict['scores']['case'] = []
                    
            fill_dict['scores']['geofips'].append(args_dict['geofips'])
            fill_dict['scores']['industry'].append(args_dict['industry'])
            fill_dict['scores']['case'].append(key)
            
            for subkey in args_dict[key]:
                if subkey not in fill_dict['scores']:
                    fill_dict['scores'][subkey] = []
                fill_dict['scores'][subkey].append(args_dict[key][subkey])
        
        else:
            fill_dict['attributes'][key] = [args_dict[key]]
            
    for entry in fill_dict:
        df_dict[entry] = pd.DataFrame(fill_dict[entry])
        print entry
        print df_dict[entry]
            
                
                                
        
    
dict_to_dfs(manf[0])

Xproj_full
X_full
ypred_test
scaled_full_metrics
X_train
train_metrics
yproj_full
ypred_train
window
scaled_train_metrics
Y_train
Y_full
X_test
ypred_full
yproj_train
Xproj_train
geofips
scaled_test_metrics
full_metrics
test_metrics
comp_time_sec
ntest
industry
Y_test
Xproj_full
  geofips industry  val0  val1  val2  val3  val4  val5  val6  val7  val8  val9
0   16940     manf  2014  2015  2016  2017  2018  2019  2020  2021  2022  2023
X_full
  geofips industry  val0  val1  val10  val11  val12  val13  val14  val15  \
0   16940     manf  1969  1970   1979   1980   1981   1982   1983   1984   

   ...   val40  val41  val42  val43  val44  val5  val6  val7  val8  val9  
0  ...    2009   2010   2011   2012   2013  1974  1975  1976  1977  1978  

[1 rows x 47 columns]
X_train
  geofips industry  val0  val1  val10  val11  val12  val13  val14  val15  \
0   16940     manf  1969  1970   1979   1980   1981   1982   1983   1984   

   ...   val31  val32  val33  val34  val4  val5  val6  val7  val8  v

ValueError: arrays must all be same length

In [35]:
test_df = pd.DataFrame({'test':1,'another':4})

print test_df

ValueError: If using all scalar values, you must pass an index

In [ ]:
def read_file_write_db(args_dict, city, geo):
    try:
        print 'Connecting to database...'
        # con = mdb.connect('localhost', 'danielj', '', 'ecotest')
        engine = create_engine(
            "mysql+mysqldb://danielj:@localhost/ecotest")
    except:
        print 'Error connecting to db:', sys.exc_info()[0]
        return 0
    try:
        print 'Writing to database...'
        df.to_sql(con=engine, name=key,
                  if_exists='replace', flavor='mysql')
    except OperationalError as err:
        print 'Error writing to db:', sys.exc_info()[0]
        print err.message
        return 0